## Tools for Building Henderson's Mixed Model Equations for Multiple Trait Model

Adding markers with random effects in multiple trait model. Added estimation of covariance matrix of marker effects.


In [1]:
using PedModule
using DataFrames
using Distributions

In [2]:
function Gibbs(A,x,b,nIter;outFreq=100)
    n = size(x,1)
    xMean = zeros(n)
    for iter = 1:nIter
        if iter%outFreq==0
            println("at sample: ",iter)
        end
        for i=1:n
            cVar = 1.0/A[i,i]
            cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
            x[i]    = randn()*sqrt(cVar) + cMean 
        end
        xMean += (x - xMean)/iter
    end
    return xMean
end

function Gibbs(A,x,b)
    n = size(x,1)
    for i=1:n
        cVar = 1.0/A[i,i]
        cMean   = cVar*(b[i] - A[:,i]'x)[1,1] + x[i]
        x[i]    = randn()*sqrt(cVar) + cMean 
    end
end

function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[:,i]'x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        #println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end

Jacobi (generic function with 1 method)

In [3]:
type TermStrVal
    str::AbstractString
    value::Float64
end

type TermLvlVal
    level::AbstractString
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::AbstractString
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{AbstractString,1}            # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type ResVar
    R0::Array{Float64,2}
    RiDict::Dict{BitArray{1},Array{Float64,2}}
end   

type MarkerMatrix
    X::Array{Float64,2}
    xArray::Array{Array{Float64,1},1}
    markerMeans::Array{Float64,2}
    mean2pq::Float64
    G::Array{Float64,2}
    centered::Bool
    function MarkerMatrix(X::Array{Float64,2},G::Array{Float64,2})
        markerMeans = center!(X) #centering
        p           = markerMeans/2.0
        mean2pq     = (2*p*(1-p)')[1,1]
        xArray      = get_column_ref(X)
        new(X,xArray,markerMeans,mean2pq,G,true)
    end
end

type MME
    modelVec::Array{AbstractString,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{AbstractString,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{AbstractString,1}
    X
    ySparse
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    R::Array{Float64,2}
    Ai
    mmePos::Int64
    missingPattern
    resVar
    M
end

In [4]:
function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getRi(resVar::ResVar,sel::BitArray{1})
    if haskey(resVar.RiDict,sel)
        return resVar.RiDict[sel]
    end
    n = size(resVar.R0,1)
    RZ = zeros(n,n)
    RZ[sel,sel] = inv(resVar.R0[sel,sel])
    resVar.RiDict[sel] = copy(RZ)
    return RZ
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,string(m)*":"*trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::AbstractString,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],keep=false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{AbstractString,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec;symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms; mTrms]
        for (i,trm) = enumerate(modelTerms) 
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,0,0,Array(Float64,1,1),R,0,1,0,0,0)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(AbstractString,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = round(Int64,[mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = round(Int64,[dict[i] for i in trm.str]) 
    end
    xi    = (trm.iModel-1)*nObs + collect(1:nObs)
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi;ii]
            xj = [xj;jj]
            xv = [xv;vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi;1;nObs*nModels]
    xj = [xj;1;1]
    xv = [xv;0;0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function mkRi(mme::MME,df::DataFrame)
    resVar = ResVar(mme.R,Dict())
    tstMsng = !isna(df[mme.lhsVec[1]])
    for i=2:size(mme.lhsVec,1)
        tstMsng = [tstMsng !isna(df[mme.lhsVec[i]])]
    end
    mme.missingPattern = tstMsng
    n    = size(tstMsng,2)
    nObs = size(tstMsng,1)
    ii = Array(Int64,nObs*n^2)
    jj = Array(Int64,nObs*n^2)
    vv = Array(Float64,nObs*n^2)
    pos = 1
    for i=1:size(tstMsng,1)
        sel = reshape(tstMsng[i,:],n)
        Ri  = getRi(resVar,sel)
        for ti=1:n
            tii = (ti-1)*nObs + i
            for tj=1:n
                tjj = (tj-1)*nObs + i
                ii[pos] = tii
                jj[pos] = tjj
                vv[pos] = Ri[ti,tj]
                pos += 1
            end
        end         
    end
    mme.resVar = resVar
    return sparse(ii,jj,vv)
end

function getMME(mme::MME, df::DataFrame)
    mme.mmePos = 1
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = convert(Array,df[mme.lhsVec[1]],0.0)
    for i=2:size(mme.lhsVec,1)
        y    = [y; convert(Array,df[mme.lhsVec[i]],0.0)] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = mkRi(mme,df)
    mme.X = X
    mme.ySparse = ySparse 
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
    if mme.ped != 0
        ii,jj,vv = PedModule.HAi(mme.ped)
        HAi = sparse(ii,jj,vv)
        mme.Ai = HAi'HAi
        addA(mme)
    end   
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::AbstractString)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::AbstractString,ped::PedModule.Pedigree, G::Array{Float64,2})
    pedTrmVec = split(randomStr," ",keep=false)
    res = []
    for trm in pedTrmVec
        for (m,model) = enumerate(mme.modelVec)
            strVec  = split(model,['=','+'])
            strpVec = [strip(i) for i in strVec]
            if trm in strpVec
                res = [res;string(m)*":"*trm]
            end
        end
    end
    mme.pedTrmVec = res
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

function sampleMissingResiduals(mme,resVec)
    msngPtrn = mme.missingPattern
    n,k = size(msngPtrn)
    yIndex = collect(0:k-1)*n
    allTrue = fill(true,k)
    for i=1:n
        notMsng = reshape(msngPtrn[i,:,],k)
        if (notMsng!=allTrue)
            msng    = !notMsng
            nMsng   = sum(msng)
            resi    = resVec[yIndex+i][notMsng]
            Ri      = mme.resVar.RiDict[notMsng][notMsng,notMsng]
            Rc      = mme.R[msng,notMsng]
            L       = chol(mme.R[msng,msng] - Rc*Ri*Rc')'
            resVec[(yIndex+i)[msng]] = Rc*Ri*resi + L*randn(nMsng) 
        end
    end
end

sampleMissingResiduals (generic function with 1 method)

In [5]:
ped = PedModule.mkPed("sim.ped");

In [6]:
dfGen = readtable("sim.gen", separator = ' ');

In [32]:
srand(1234)
Q = convert(Array{Float64,2},dfGen[:,collect(2:end)]);
α1 = randn(200)
α2 = randn(200) + 0.5α1
a1 = Q*α1
a2 = Q*α2;

In [33]:
R = cov([a1 a2])

2x2 Array{Float64,2}:
 58.4077  24.6024
 24.6024  67.2585

In [81]:
GM  = R./200

2x2 Array{Float64,2}:
 0.292038  0.123012
 0.123012  0.336293

In [80]:
R0 = R
L  = chol(R0)
e  = L'*randn(2,size(Q,1))
y = [a1 a2] + e';

In [39]:
cov(e')

2x2 Array{Float64,2}:
 56.8318  23.763 
 23.763   66.7894

In [57]:
cov(y)

2x2 Array{Float64,2}:
 114.822    48.4003
  48.4003  134.101 

In [40]:
df2 = DataFrame(Animal = dfGen[:,1], y1=y[:,1],y2=y[:,2]);

In [41]:
head(df2)

,Animal,y1,y2
1,10102,5.919949392746034,0.7235531261963493
2,10103,-17.36969088823628,11.610884248971198
3,10104,-2.370795821095914,11.591462275367984
4,10105,-4.562728519081498,3.593927517577562
5,10106,-3.7714944777212143,-13.901146146503452
6,10107,20.87498656890901,-2.8062396106490928


In [42]:
models = "y1 = intercept ;
          y2 = intercept "
mme = initMME(models,R)
nothing

##### functions for markers

In [43]:
function initVec(a::Float64,x::Array{Float64,1})
    for v in x
        v = a
    end
end

function get_column(X,j)
    nrow,ncol = size(X)
    if j>ncol||j<0
        error("column number is wrong!")
    end
    indx = 1 + (j-1)*nrow
    ptr = pointer(X,indx)
    pointer_to_array(ptr,nrow)
end

function get_column_ref(X)
    ncol = size(X)[2]
    xArray = Array(Array{Float64,1},ncol)
    for i=1:ncol
        xArray[i] = get_column(X,i)
    end
    return xArray
end

function center!(X)
    nrow,ncol = size(X)
    colMeans = mean(X,1)
    BLAS.axpy!(-1,ones(nrow)*colMeans,X)
    return colMeans
end

function sample_effects_ycorr!(xArray,xpx,yCorr,alphaArray,meanAlpha,invR0,invG0,iIter,traiti) 
    # function to sample effects for trait i
    # invR0 is inverse of R0[traiti,traiti]
    # invG0 is inverse of G0
    
    nEffects = length(xArray)
    α = alphaArray[traiti] # vector of effects for traiti across all loci
    nTraits = length(alphaArray)
    β = zeros(nTraits)     # vector of effects for locus j across all traits
    for j=1:nEffects
        for k=1:nTraits
            β[k] = alphaArray[k][j]
        end
        x        = xArray[j]
        lhs      = xpx[j]*invR0 + invG0[traiti,traiti]
        rhs      = dot(x,yCorr) - (invG0[traiti,:]*β)[1]
        invLhs   = 1.0/lhs
        mean     = invLhs*rhs + α[j]
        oldAlpha = α[j]
        α[j]     = mean + randn()*sqrt(invLhs)
        BLAS.axpy!((oldAlpha-α[j,1])*invR0,x,yCorr)
        meanAlpha[j] += (α[j] - meanAlpha[j])/iIter
    end
end


sample_effects_ycorr! (generic function with 1 method)

### Adding marker information for multiple-traits

Let 
$$
\mathbf{w}_i = \mathbf{y}_i - \mathbf{W}_i\mathbf{\theta}_i - \mathbf{M}\mathbf{\theta}_i,
$$

where $\mathbf{\theta}_i$ are the effects in the mixed model for trait $i$, $\mathbf{W}_i$ is the incidnce matrix for these effects, $\mathbf{M}$ are the marker covariates, and $\mathbf{\theta}_i$ are their effects for trait $i$. Then "sample_effects_ycorr!" is called to sample $\mathbf{\alpha}_i$ for each trait $i$ with "yCorr" set to
$$
\text{yCorr} = \sum_j r^{ij}\mathbf{w}_i,
$$
where $r^{ij}$ is the $ij$th element from $\mathbf{R}_0^{-1}$. After sampling $\mathbf{\alpha}_i$, $\mathbf{W}_i$ is updated as

$$
\mathbf{w}_i = \mathbf{w}_i + \mathbf{M}(\mathbf{\theta}_i^{\text{old}} - \mathbf{\theta}_i^{\text{new}}).
$$

In [44]:
mme.M = MarkerMatrix(Q,GM); 

In [77]:
function sampleMCMC(nIter,mme,df;outFreq=100)
    getMME(mme,df)
    p = size(mme.mmeLhs,1)
    nMarkers = size(mme.M.X,2)
    sol = fill(0.0,p)
    solMean = fill(0.0,p)
    GaussSeidel(mme.mmeLhs,sol,mme.mmeRhs,tol=0.000001) 
    ν = 10
    nObs    = size(df,1)
    nTraits = size(mme.lhsVec,1)
    νR0 = ν + nTraits
    R0 = mme.R
    PRes = R0*(νR0 - nTraits - 1)
    SRes   = zeros(Float64,nTraits,nTraits)
    R0Mean = zeros(Float64,nTraits,nTraits)
    if mme.ped != 0
        pedTrmVec = mme.pedTrmVec
        k   = size(pedTrmVec,1)        
        νG0 = ν + k
        G0  = inv(mme.Gi)
        P   = G0*(νG0 - k - 1)
        S   = zeros(Float64,k,k)
        G0Mean = zeros(Float64,k,k)
    end
    νGM = 4.0 + nTraits
    PM  = mme.M.G*(νGM - nTraits - 1)
    SM  = zeros(Float64,nTraits,nTraits) 
    GMMean = zeros(Float64,nTraits,nTraits) 
    
    mArray = mme.M.xArray
    mpm    = [dot(mme.M.X[:,i],mme.M.X[:,i]) for i=1:size(mme.M.X,2)]   
    ycorr  = vec(full(mme.ySparse))
    wArray         = Array(Array{Float64,1},nTraits)
    alphaArray     = Array(Array{Float64,1},nTraits)
    meanAlphaArray = Array(Array{Float64,1},nTraits)
    for traiti = 1:nTraits
        startPosi = (traiti-1)*nObs  + 1
        ptr = pointer(ycorr,startPosi) 
        wArray[traiti] = pointer_to_array(ptr,nObs) 
        alphaArray[traiti]     = zeros(nMarkers)
        meanAlphaArray[traiti] = zeros(nMarkers)
    end
    yCorr  = zeros(nObs)
    temp1  = zeros(nMarkers)
    temp2  = zeros(p)
########################################################################################    
    # loop for MCMC sampling
    for iter=1:nIter
        if iter%outFreq==0
            println("at sample: ",iter,"\n")
        end
        # sample effects included in mme.model
        Gibbs(mme.mmeLhs,sol,mme.mmeRhs)
        ycorr[:] = ycorr[:] - mme.X*sol 
        # sample marker effects for each traiti
        iR0 = inv(mme.R)
        iGM = inv(mme.M.G)
        for traiti = 1:nTraits # sample alpha_i for each trait
            initVec(0.0,yCorr)
            for traitj =1:nTraits # making yCorr for trait i
                yCorr[:] = yCorr[:] + iR0[traiti,traitj]*wArray[traitj]
            end
            #sample marker effetcs for trait 
            oldAlpha = copy(alphaArray[traiti])
            sample_effects_ycorr!(mArray,mpm,yCorr,alphaArray,meanAlphaArray[traiti],iR0[traiti,traiti],iGM,iter,traiti)
            # update w_i
            wArray[traiti][:] = wArray[traiti][:] + mme.M.X*(oldAlpha - alphaArray[traiti])
        end
        # can make this more efficient by taking advantage of symmetry
        if mme.ped != 0
            for (i,trmi) = enumerate(pedTrmVec)    
                pedTrmi  = mme.modelTermDict[trmi]
                startPosi  = pedTrmi.startPos
                endPosi    = startPosi + pedTrmi.nLevels - 1
                for (j,trmj) = enumerate(pedTrmVec)
                    pedTrmj  = mme.modelTermDict[trmj]
                    startPosj  = pedTrmj.startPos
                    endPosj    = startPosj + pedTrmj.nLevels - 1
                    S[i,j] = (sol[startPosi:endPosi]'*mme.Ai*sol[startPosj:endPosj])[1,1]
                end
            end
        end
        resVec = ycorr
        #sampleMissingResiduals(mme,resVec)
        for traiti = 1:nTraits
            startPosi = (traiti-1)*nObs + 1
            endPosi   = startPosi + nObs - 1
            for traitj = traiti:nTraits
                startPosj = (traitj-1)*nObs + 1
                endPosj   = startPosj + nObs - 1
                SRes[traiti,traitj] = (resVec[startPosi:endPosi]'resVec[startPosj:endPosj])[1] 
                SRes[traitj,traiti] = SRes[traiti,traitj]
                SM[traiti,traitj] = (alphaArray[traiti]'alphaArray[traitj])[1]
                SM[traitj,traiti] = SM[traiti,traitj]
            end
        end
        mme.M.G = rand(InverseWishart(νGM + nMarkers, PM + SM))
        R0 = rand(InverseWishart(νR0 + nObs, PRes + SRes))
        mme.R = R0
        Ri = mkRi(mme,df)
        X = mme.X
        mme.mmeLhs = X'Ri*X
        ycorr[:] = ycorr[:] + mme.X*sol
        mme.mmeRhs = mme.X'Ri*ycorr
        if mme.ped != 0
            pedTrm1 = mme.modelTermDict[pedTrmVec[1]]
            q = pedTrm1.nLevels
            G0 = rand(InverseWishart(νG0 + q, P + S))
            mme.Gi = inv(G0)
            addA(mme)
            G0Mean  += (G0  - G0Mean )/iter     
        end
        solMean += (sol - solMean)/iter 
        R0Mean  += (R0  - R0Mean )/iter
        GMMean  += (mme.M.G  - GMMean)/iter
    end
    output = Dict()
    output["posteriorMeanLocationParms"] = solMean
    output["posteriorMeanAlpha"] = meanAlphaArray
    output["posteriorMeanR0"] = R0Mean
    output["posteriorMeanGM"] = GMMean
    return output
end

sampleMCMC (generic function with 1 method)

In [58]:
R0

2x2 Array{Float64,2}:
 58.4077  24.6024
 24.6024  67.2585

In [78]:
@time res = sampleMCMC(300,mme,df2)
res["posteriorMeanR0"]

at sample: 100

at sample: 200

at sample: 300

2x2 Array{Float64,2}:
 57.2108  21.9573
 21.9573  68.1568



  7.175024 seconds (60.18 M allocations: 2.356 GB, 4.29% gc time)


In [67]:
mean(y,1)

1x2 Array{Float64,2}:
 4.43083  2.89755

In [68]:
res["posteriorMeanLocationParms"]

2-element Array{Float64,1}:
 4.42186
 2.90324

In [69]:
cor([Q*res["posteriorMeanAlpha"][1] Q*α1])

2x2 Array{Float64,2}:
 1.0       0.977099
 0.977099  1.0     

In [82]:
cov([α1 α2])

2x2 Array{Float64,2}:
 1.05775   0.585118
 0.585118  1.19751 

In [79]:
res["posteriorMeanGM"]

2x2 Array{Float64,2}:
 1.20833   0.777518
 0.777518  1.39265 